In [1]:
import numpy as np
import pandas as pd
import scipy.stats as ss
import matplotlib.pyplot as plt
from scipy.integrate import quad
from scipy.stats import norm


In [2]:
rho = pd.read_pickle("SABR_Rho_full.pickle")
alpha = pd.read_pickle("SABR_Alpha_full.pickle")
nu = pd.read_pickle("SABR_Nu_full.pickle")
df = pd.read_csv('ois.csv')
forward_swap = pd.read_csv('forward_swap_6m_rates_3m.csv')

forward_swap = forward_swap[forward_swap['tenor'] != 4.0]
forward_swap = forward_swap[forward_swap['tenor'] != 6.0]
forward_swap = forward_swap[forward_swap['tenor'] != 7.0]
forward_swap = forward_swap[forward_swap['tenor'] != 8.0]
forward_swap = forward_swap[forward_swap['tenor'] != 9.0]
forward_swap

,Unnamed: 0,forward,tenor,forward_swap_rate
0,0,0.25,1.0,0.029674
1,1,0.25,2.0,0.031143
2,2,0.25,3.0,0.032345
4,4,0.25,5.0,0.033689
9,9,0.25,10.0,0.037417
...,...,...,...,...
390,390,10.00,1.0,0.042151
391,391,10.00,2.0,0.043078
392,392,10.00,3.0,0.044062
394,394,10.00,5.0,0.046210


In [3]:
sabr = pd.DataFrame()
maturities = []
tenors = []
rho_df = []
alpha_df = []
nu_df = []
beta_df = []
for i in range(len(rho.index)):
    for j in range(len(rho.columns)):
        maturity = rho.index[i]
        tenor = rho.columns[j]
        rho_val = rho[rho.columns[j]][rho.index[i]]
        alpha_val = alpha[rho.columns[j]][rho.index[i]]
        nu_val = nu[rho.columns[j]][rho.index[i]]
        beta = 0.9
        maturities.append(maturity)
        tenors.append(tenor)
        rho_df.append(rho_val)
        alpha_df.append(alpha_val)
        nu_df.append(nu_val)
        beta_df.append(beta)

sabr = pd.DataFrame({'Maturity':maturities,'Tenor':tenors,'Alpha':alpha_df,'Beta':beta_df,'Rho':rho_df,'Nu':nu_df}).set_index('Maturity')
sabr

,Tenor,Alpha,Beta,Rho,Nu
Maturity,,,,,
0.25,1,0.132085,0.9,-0.643879,2.226352
0.25,2,0.180057,0.9,-0.518660,1.842792
0.25,3,0.192705,0.9,-0.463653,1.576265
0.25,5,0.175247,0.9,-0.391128,1.181183
0.25,10,0.169857,0.9,-0.221986,0.863130
...,...,...,...,...,...
10.00,1,0.177497,0.9,-0.545390,1.006122
10.00,2,0.195300,0.9,-0.544375,0.924829
10.00,3,0.206884,0.9,-0.549852,0.867641


### Part 1:

A decompounded option pays the following at time T=5y:

$$ CMS 10y^{1/p}-0.04^{1/q} $$
$$ g(K)=K^{0.25}-0.04^{0.5} , \; g'(K)=0.25*K^{-0.75} , \; g"(K)=-0.1875K^{-1.75}$$
$$ h(K)=\frac{g(K)}{IRR(K)}=\frac{K^{0.25}-0.04^{0.5}}{IRR(K)}$$
$$ h'(K)=\frac{IRR(K)g'(K)-g(K)IRR'(K)}{IRR(K)^2}=\frac{IRR(K)*0.25*K^{-0.75} -(K^{0.25}-0.04^{0.5})IRR'(K)}{IRR(K)^2}$$
$$ h"(K)=\frac{IRR(K)g"(K)-IRR"(K)g(K)-2*IRR'(K)g'(K)}{IRR(K)^2}+\frac{2*IRR'(K)^2g(K)}{IRR(K)^3}$$
$$=\frac{IRR(K)(-0.1875K^{-1.75})-IRR"(K)(K^{0.25}-0.04^{0.5})-2*IRR'(K)(0.25*K^{-0.75})}{IRR(K)^2}+\frac{2*IRR'(K)^2(K^{0.25}-0.04^{0.5})}{IRR(K)^3}$$



In [4]:
def IRR(K,m,N):
    # implementation of IRR(K) function
    value = 1/K * ( 1.0 - 1/(1 + K/m)**(N*m) )
    return value

def IRR_d1(K,m,N):
    # implementation of IRR'(K) function (1st derivative)
    firstDerivative = -1/K*IRR(K, m, N) + 1/(K*m)*N*m/(1+K/m)**(N*m+1)
    return firstDerivative

def IRR_d2(K,m,N):
    # implementation of IRR''(K) function (2nd derivative)
    secondDerivative = -2/K*IRR_d1(K, m, N) - 1/(K*m*m)*(N*m)*(N*m+1)/(1+K/m)**(N*m+2)
    return secondDerivative

def g_p1(K):
    value = K**0.25 - 0.04**0.5
    return value

def g_p1_d1(K):
    firstDerivative = 0.25*K**(-0.75)
    return firstDerivative

def g_p1_d2(K):
    secondDerivative = -0.1875*K**(-1.75)
    return secondDerivative

def h_p1(K,m,N):
    value = (K**0.25 - 0.04**0.5) / IRR(K,m,N)
    return value

def h_p1_d1(K,m,N):
    firstDerivative = (IRR(K,m,N)*g_p1_d1(K) - g_p1(K)*IRR_d1(K,m,N)) / (IRR(K,m,N)**2)
    return firstDerivative

def h_p1_d2(K,m,N):
    part1 = (IRR(K,m,N)*g_p1_d2(K) - (IRR_d2(K,m,N)*g_p1(K)) - (2*IRR_d1(K,m,N)*g_p1_d1(K))) / (IRR(K,m,N)**2)
    part2 = (2*IRR_d1(K,m,N)**2 *g_p1(K)) / (IRR(K,m,N)**3)
    secondDerivative = part1 + part2
    return secondDerivative

def Black76Pay(F, K, m, N, T, sigma):
    d1 = (np.log(F/K)+(sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return IRR(F,m,N)*(F*norm.cdf(d1) - K*norm.cdf(d2))

def Black76Rec(F, K, m, N, T, sigma):
    d1 = (np.log(F/K)+(sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return IRR(F,m,N)*(K*norm.cdf(-d2) - F*norm.cdf(-d1))

def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    if F == K:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom
    return sabrsigma

def CMS_p1(K,m,N,F,T,alpha,beta,rho,nu,upper,DF):
    forward_part = g_p1(F)
    rec_integral = quad(lambda x: h_p1_d2(x,m,N)*DF*Black76Rec(F,x,m,N,T,SABR(F,x,T,alpha,beta,rho,nu)),0,F)
    pay_integral = quad(lambda y: h_p1_d2(y,m,N)*DF*Black76Pay(F,y,m,N,T,SABR(F,y,T,alpha,beta,rho,nu)),F,upper)
    return DF*forward_part + rec_integral[0] + pay_integral[0]

In [5]:
df_1 = forward_swap[forward_swap['tenor'] == 10]
df_1 = df_1[df_1['forward'] == 5.0]
df_1

,Unnamed: 0,forward,tenor,forward_swap_rate
199,199,5.0,10.0,0.043619


In [6]:
sabr_1 = sabr[sabr['Tenor']==10]
sabr_1 = sabr_1[sabr_1.index==5.0]
sabr_1

,Tenor,Alpha,Beta,Rho,Nu
Maturity,,,,,
5.0,10,0.176964,0.9,-0.436751,0.497116


In [7]:
df[df['Tenor'] == 5.0]

,Tenor,Tenor.1,Product,Rate,Do,fn
9,5.0,5y,OIS,0.0036,0.982184,0.003996


In [8]:
CMS_p1(0.043619,2,10,0.043619,5,0.176964,0.9,-0.436751,0.497116,5,0.982184)

C:\Users\SHERMA~1\AppData\Local\Temp/ipykernel_5236/3310306143.py:75: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  rec_integral = quad(lambda x: h_p1_d2(x,m,N)*DF*Black76Rec(F,x,m,N,T,SABR(F,x,T,alpha,beta,rho,nu)),0,F)


0.23063513511939632

### Part 2:

For a caplet, the present value is:

$$ V_0 = V^{pay}(K)h'(L)+\int_{L}^{\infty}h"(K)V^{pay}(K)dK$$

Lower boundary is when the payoff at K=L is 0. Solving the inequality we get L equals to 0.0016.

$$F^{\frac{1}{4}} - 0.04^{\frac{1}{2}} > 0$$
$$F > 0.0016 = L$$

In [10]:
def CMS_caplet_p1(K,m,N,F,T,L,alpha,beta,rho,nu,upper,DF):
    first_part = h_p1_d1(L,m,N)*DF*Black76Pay(F,L,m,N,T,SABR(F,L,T,alpha,beta,rho,nu))
    pay_integral = quad(lambda x: h_p1_d2(x,m,N)*DF*Black76Pay(F,x,m,N,T,SABR(F,x,T,alpha,beta,rho,nu)),L,upper)
    return first_part + pay_integral[0]

In [11]:
CMS_caplet_p1(0.043619,2,10,0.043619,5,0.0016,0.176964,0.9,-0.436751,0.497116,5,0.982184)
    

0.24392138361733284